ДЗ Гибридные рекомендательные системы
=====
1.Датасет ml-latest
2.Вспомнить подходы, которые мы разбирали
3.Выбрать понравившийся подход к гибридным системам
4.Написать свою

In [164]:
import pandas as pd
import numpy as np
from surprise import SVD, SVDpp
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [15]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')

In [27]:
movies_ratings = pd.merge(ratings, movies, on='movieId', how='inner')

In [28]:
movies_ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100836 entries, 0 to 100835
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   userId     100836 non-null  int64  
 1   movieId    100836 non-null  int64  
 2   rating     100836 non-null  float64
 3   timestamp  100836 non-null  int64  
 4   title      100836 non-null  object 
 5   genres     100836 non-null  object 
dtypes: float64(1), int64(3), object(2)
memory usage: 5.4+ MB


In [29]:
['Ok' for x in movies_ratings.isnull().any() if x == False ]

['Ok', 'Ok', 'Ok', 'Ok', 'Ok', 'Ok']

In [30]:
del movies_ratings['timestamp']
movies_ratings.head()

,userId,movieId,rating,title,genres
0,1,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy


In [31]:
dataset = pd.DataFrame({
    'uid': movies_ratings.userId,
    'iid': movies_ratings.title,
    'rating': movies_ratings.rating
})
dataset.head()

,uid,iid,rating
0,1,Toy Story (1995),4.0
1,5,Toy Story (1995),4.0
2,7,Toy Story (1995),4.5
3,15,Toy Story (1995),2.5
4,17,Toy Story (1995),4.5


Создание пользователя
=======

In [99]:
np.max(movies_ratings.userId)

610

In [101]:
# создадим нового пользователя
new_user_id = 700

In [102]:
random_films = movies_ratings.title.sample(10)

In [103]:
random_films

84027                                Over the Hedge (2006)
40015                                       Witness (1985)
7150     Raiders of the Lost Ark (Indiana Jones and the...
34228                     Hunt for Red October, The (1990)
22996                                Eyes Wide Shut (1999)
78846                                          Argo (2012)
43069                                         Brave (2012)
93011                           Stepford Wives, The (1975)
43427                                       Spectre (2015)
37404                              X2: X-Men United (2003)
Name: title, dtype: object

In [104]:
index_movies = random_films.index

In [105]:
new_user_ratings = {
    random_films[index_movies[0]]: 3.5,
    random_films[index_movies[1]]: 2.5,
    random_films[index_movies[2]]: 4.5,
    random_films[index_movies[3]]: 2.5,
    random_films[index_movies[4]]: 5.0,
    random_films[index_movies[5]]: 5.0,
    random_films[index_movies[6]]: 4.0,
    random_films[index_movies[7]]: 3.0,
    random_films[index_movies[8]]: 2.0,
    random_films[index_movies[9]]: 1.0
}

In [106]:
new_user_ratings

{'Over the Hedge (2006)': 3.5,
 'Witness (1985)': 2.5,
 'Raiders of the Lost Ark (Indiana Jones and the Raiders of the Lost Ark) (1981)': 4.5,
 'Hunt for Red October, The (1990)': 2.5,
 'Eyes Wide Shut (1999)': 5.0,
 'Argo (2012)': 5.0,
 'Brave (2012)': 4.0,
 'Stepford Wives, The (1975)': 3.0,
 'Spectre (2015)': 2.0,
 'X2: X-Men United (2003)': 1.0}

Добавим новые фильмы в дата-сет

In [108]:
for film, rating in new_user_ratings.items(): 
    dataset = dataset.append({
            'uid': new_user_id,
            'iid': film,
            'rating': rating
        }, ignore_index=True)


In [110]:
dataset.tail(15)

,uid,iid,rating
100831,610,Bloodmoon (1997),2.5
100832,610,Sympathy for the Underdog (1971),4.5
100833,610,Hazard (2005),3.0
100834,610,Blair Witch (2016),3.5
100835,610,31 (2016),3.5
100836,700,Over the Hedge (2006),3.5
100837,700,Witness (1985),2.5
100838,700,Raiders of the Lost Ark (Indiana Jones and the...,4.5
100839,700,"Hunt for Red October, The (1990)",2.5
100840,700,Eyes Wide Shut (1999),5.0


Используем два алгоритма
=======

In [111]:
reader = Reader(rating_scale=(ratings.rating.min(), ratings.rating.max()))

In [112]:
data = Dataset.load_from_df(dataset, reader)

SVDpp
======

In [ ]:
svdpp = SVDpp()
trainset = data.build_full_trainset()
svdpp.fit(trainset)

KNNWithMeans
======

In [115]:
knn_m = KNNWithMeans(k=30 , sim_options={'name': 'pearson_baseline', 'user_based': False})
knn_m.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


Блендинг
=====

In [118]:
#список фильмов для рекомендации
num_films_recommendations = 20

Предскажем рейтинг фильма двумя алгоритмами

In [122]:
new_user_predict_svdpp = {}

for movie in movies.title.unique():
    if movie in new_user_ratings:
        continue
    new_user_predict_svdpp[movie] = svdpp.predict(uid=new_user_id, iid=movie)

In [123]:
new_user_predict_knn = {}

for movie in movies.title.unique():
    if movie in new_user_ratings:
        continue
    new_user_predict_knn[movie] = knn_m.predict(uid=new_user_id, iid=movie)

Вычислим оценку фильма с весами 0.7 и 0.3

In [157]:
blended_user_predict = {}

for movie_svdpp in new_user_predict_svdpp.items():
    blended_user_predict[movie_svdpp[0]] = movie_svdpp[1].est *0.7 + new_user_predict_knn[movie_svdpp[0]].est * 0.3
    #print(movie_svdpp[0])
    #print(movie_svdpp[1].est)
    #print(new_user_predict_knn[movie_svdpp[0]].est)

In [160]:
blended_recommendations = sorted(blended_user_predict.items(), key=lambda x: x[1], reverse=True)[:num_films_recommendations]

In [167]:
print('Recommendations')
print('_'*20)
blended_recommendations

Recommendations
____________________


[('Fight Club (1999)', 4.603382117367926),
 ('Shawshank Redemption, The (1994)', 4.459695114179679),
 ('Brazil (1985)', 4.441588686732177),
 ('Hustler, The (1961)', 4.438544721980346),
 ('Fear and Loathing in Las Vegas (1998)', 4.428993426297985),
 ('Trainspotting (1996)', 4.3841312768443395),
 ('Usual Suspects, The (1995)', 4.372903527956951),
 ('Touch of Evil (1958)', 4.36475635746868),
 ('North by Northwest (1959)', 4.342465080227652),
 ('Rear Window (1954)', 4.339536048384625),
 ('Seven Samurai (Shichinin no samurai) (1954)', 4.338572446936224),
 ('Hoop Dreams (1994)', 4.3312013654974875),
 ('Pulp Fiction (1994)', 4.317704218384147),
 ('There Will Be Blood (2007)', 4.2831571732722376),
 ('Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 4.281778962975426),
 ('Seventh Seal, The (Sjunde inseglet, Det) (1957)', 4.273637027369897),
 ("No Man's Land (2001)", 4.264562287818652),
 ('Intouchables (2011)', 4.264022119099932),
 ('Underground (1995)', 4.259208533191971),
 ('Kolya (Kolja) (199